<a href="https://colab.research.google.com/github/Paunty/SeaWex/blob/main/tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to Train YOLOv5 on Custom Objects for Seeed Vision product

This tutorial is based on the [YOLOv5-swift repository](https://github.com/Seeed-Studio/yolov5-swift) by [Seeed-Studio](https://www.seeedstudio.com/). This notebook shows training on **your own custom objects**. Many thanks to USeeed-Studio for putting this repository together - we hope that in combination with clean data management tools at Roboflow, this technologoy will become easily accessible to any developer wishing to use computer vision in their projects.

### Accompanying Blog Post

We recommend that you follow along in this notebook while reading the blog post on [how to train YOLOv5](https://blog.roboflow.ai/how-to-train-yolov5-on-a-custom-dataset/), concurrently.

### Steps Covered in this Tutorial

In this tutorial, we will walk through the steps required to train YOLOv5 on your custom objects. We use a [public blood cell detection dataset](https://public.roboflow.ai/object-detection/bccd), which is open source and free to use. You can also use this notebook on your own data.

To train our detector we take the following steps:

* Install YOLOv5 dependencies
* Download custom YOLOv5 object detection data
* Write our YOLOv5 Training configuration
* Run YOLOv5 training
* Evaluate YOLOv5 performance
* Visualize YOLOv5 training data
* Run YOLOv5 inference on test images
* Export saved YOLOv5 weights for future inference



### **About**

[Roboflow](https://roboflow.com) enables teams to deploy custom computer vision models quickly and accurately. Convert data from to annotation format, assess dataset health, preprocess, augment, and more. It's free for your first 1000 source images.

**Looking for a vision model available via API without hassle? Try Roboflow Train.**

![Roboflow Wordmark](https://files.seeedstudio.com/wiki/SenseCAP-A1101/57.png)



# Install Dependencies

_(Remember to choose GPU in Runtime if not already selected. Runtime --> Change Runtime Type --> Hardware accelerator --> GPU)_

In [3]:
#començam de 0
# Install Roboflow to download dataset
!pip install -q roboflow

# Import Roboflow
from roboflow import Roboflow

rf = Roboflow(api_key="19k0ofVB2Xzm912LeKDg")
project = rf.workspace("ia-barcos").project("boat-detection-sli2d")
dataset = project.version(2).download("folder")

# Verify that the dataset is correctly downloaded
!ls /content/




loading Roboflow workspace...
loading Roboflow project...
Boat-Detection-2  sample_data


In [4]:
# Setup Yolov5 Classification
# Remove any existing YOLOv5 directory (especially if it's YOLOv5-Swift)
!rm -rf yolov5

# Clone the official YOLOv5 repository from Ultralytics (supports classification)
!git clone https://github.com/ultralytics/yolov5
%cd yolov5

# Install YOLOv5 dependencies
!pip install -r requirements.txt

# Verify that YOLOv5 is properly installed
!python -c "import torch; print('Torch version:', torch.__version__)"



Cloning into 'yolov5'...
remote: Enumerating objects: 17270, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 17270 (delta 0), reused 0 (delta 0), pack-reused 17269 (from 2)
Receiving objects: 100% (17270/17270), 16.11 MiB | 29.45 MiB/s, done.
Resolving deltas: 100% (11861/11861), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Get Correctly Formatted Custom Dataset [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/roboflow-ai/yolov5-custom-training-tutorial/blob/main/yolov5-custom-training.ipynb)
We'll download our dataset from Roboflow. Use the "**YOLOv5 PyTorch**" export format. Note that the Ultralytics implementation calls for a YAML file defining where your training and test data is. The Roboflow export also writes this format for us.

To get your data into Roboflow, follow the [Getting Started Guide](https://blog.roboflow.ai/getting-started-with-roboflow/).
<p align=""><a href="https://roboflow.com/?ref=ultralytics"><img width="480" src="https://uploads-ssl.webflow.com/5f6bc60e665f54545a1e52a5/6152a275ad4b4ac20cd2e21a_roboflow-annotate.gif"/></a></p>Label images lightning fast (including with model-assisted labeling)

In [5]:
#3️⃣ Organize Dataset for YOLOv5 Classification

# Move dataset into the correct directory if needed
!mv /content/Boat-Detection-2 /content/Boat-Classification

# Verify dataset structure
!ls /content/Boat-Classification


#4️⃣ Train YOLOv5 in Classification Mode
# Train YOLOv5 for image classification
!python classify/train.py --model yolov5s-cls.pt --data /content/Boat-Classification --epochs 50 --img 224

# Verify that the model file was created
!ls runs/train-cls/exp/weights/



README.roboflow.txt  test  train  valid
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-02-17 20:46:52.177658: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739825212.446861    1998 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739825212.517198    1998 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting 

In [6]:
!ls /content/Boat-Classification
!ls /content/Boat-Classification/train
!ls /content/Boat-Classification/valid


README.roboflow.txt  test  train  valid
 free   motora	'sailing boat'
 motora  'sailing boat'


In [ ]:
# this is the YAML file Roboflow wrote for us that we're loading into this notebook with our data
%cat {dataset.location}/data.yaml

# Evaluate Custom YOLOv5 Detector Performance

Training losses and performance metrics are saved to Tensorboard and also to a logfile defined above with the **--name** flag when we train. In our case, we named this `yolov5s_results`. (If given no name, it defaults to `results.txt`.) The results file is plotted as a png after training completes.

Note from Glenn: Partially completed `results.txt` files can be plotted with `from utils.utils import plot_results; plot_results()`.

In [ ]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
# we can also output some older school graphs if the tensor board isn't working for whatever reason...
from utils.plots import plot_results  # plot results.txt as results.png
Image(filename='/content/yolov5-swift/runs/train/yolov5n6_results/results.png', width=1000)  # view results.png

### Curious? Visualize Our Training Data with Labels

After training starts, view `train*.jpg` images to see training images, labels and augmentation effects.

Note a mosaic dataloader is used for training (shown below), a new dataloading concept developed by Glenn Jocher and first featured in [YOLOv4](https://arxiv.org/abs/2004.10934).

In [ ]:
# first, display our ground truth data
print("GROUND TRUTH TRAINING DATA:")
Image(filename='/content/yolov5-swift/runs/train/yolov5n6_results/val_batch0_labels.jpg', width=900)

In [ ]:
# print out an augmented training example
print("GROUND TRUTH AUGMENTED TRAINING DATA:")
Image(filename='/content/yolov5-swift/runs/train/yolov5n6_results/val_batch0_pred.jpg', width=900)

# Run Inference  With Trained Weights
Run inference with a pretrained checkpoint on contents of `test/images` folder downloaded from Roboflow.

In [ ]:
# trained weights are saved by default in our weights folder
%ls runs/

In [ ]:
%ls runs/train/yolov5n6_results/weights

In [ ]:
# when we ran this, we saw .007 second inference time. That is 140 FPS on a TESLA P100!
# use the best weights!
%cd /content/yolov5-swift/
!python detect.py --weights runs/train/yolov5n6_results/weights/best.pt --img 320 --conf 0.6 --source ./data/images --name test

In [ ]:
#display inference on ALL test images
#this looks much better with longer training above

import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5-swift/runs/detect/test/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

[link text](https://)# Export tflite file

In [ ]:
%cd /content/yolov5-swift/
!python export.py --data {dataset.location}/data.yaml --weights runs/train/yolov5n6_results/weights/best.pt --imgsz 192 --int8 --include tflite

# tflite to uf2

# GROVEAI For Grove Vision AI and Vision AI for SenseCAP A1101
# !python uf2conv.py -f GROVEAI -t 1 -c runs//train/yolov5n6_results//weights/best-int8.tflite -o best.uf2
# !python uf2conv.py -f VISIONAI  -t 1 -c runs//train/yolov5n6_results//weights/best-int8.tflite -o best.uf2

In [ ]:

!python uf2conv.py -f VISIONAI  -t 1 -c runs//train/yolov5n6_results//weights/best-int8.tflite -o best.uf2
%cp best.uf2 ../

**bold text**# Download Trained model for Future Inference

Now that you have trained your custom detector, you can export the trained model you have made here for inference on your device elsewhere

In [ ]:
'print("Download your custom model")'

# Please download the trained weight files in the following order
- Find the file option in the menu to the right of the colab
  
  ![download_1.png](https://github.com/Seeed-Studio/yolov5-swift/blob/master/extra/download_1.png?raw=true)

- Select the .uf2 file generated above
  
  ![download_2.png](https://github.com/Seeed-Studio/yolov5-swift/blob/master/extra/download_2.png?raw=true)

- Click the right mouse button and select Download
  
  ![download_3.png](https://github.com/Seeed-Studio/yolov5-swift/blob/master/extra/download_3.png?raw=true)
  
  



# Deploy Your Model to Grove AI Devices
- Install the latest version of [Google Chrome](https://www.google.com/chrome/) or [Microsoft Edge](https://www.microsoft.com/edge) browser, and open it.

- Plug your Grove AI device into your pc.
- Download Model into your Grove AI device
  - Double-click the boot button

    You can see a GROVEBOOT label on your pc that can move the drive.
    ![deploy_1.png](https://github.com/Seeed-Studio/yolov5-swift/blob/master/extra/deploy_1.png?raw=true)

  - Drag and drop the .uf2 file you downloaded to your GROVEBOOT removable disk
  - Open [grove ai vision preview](https://files.seeedstudio.com/grove_ai_vision/index.html)
    - Click Connect Button and select Grove AI Camera.
    ![deploy_2.png](https://github.com/Seeed-Studio/yolov5-swift/blob/master/extra/deploy_2.png?raw=true)

  - View real-time speculative results through the preview window
    ![deploy_3.png](https://github.com/Seeed-Studio/yolov5-swift/blob/master/extra/deploy_3.png?raw=true)

# Congrats!

Hope you enjoyed this!

--Team Seeed Studio